###### Voronoi constructor Module 

### Usefull Libraries

In [6]:
import math
import numpy as np
import sys

import matplotlib.pyplot as plt
import matplotlib.path as path
import matplotlib.colors as mcolors
import matplotlib._color_data as mcd
from matplotlib.patches import Polygon as pltpolygon
from matplotlib.collections import PatchCollection

from scipy.spatial import Voronoi, voronoi_plot_2d, Delaunay

from shapely.geometry import LineString, Polygon, MultiPolygon, Point
from shapely.ops import polygonize, unary_union

import import_ipynb
import node_linker
import pps
import utils

### Deadlock Function

`def deadlock(pos, bvc, parameters, robots_path, deadlock_pointers, verbose)`

Check if there are robots in deadlock situation

#### Parameters:
* **pos&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;dictionary**
    * Robots positions and distances
* **bvc&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;dictionary**
    * Information of BVC
* **parameters&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;dictionary**
    * Global parameters
* **robots_path&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;list of ndarryas of list of floats**
    * Previous positions
* **deadlock_pointers&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of ints**
    * Pointers to robots in deadlock
* **verbose&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;boolean**
    * Flag for verbose
    
#### Return:
* **fixed_up_positions&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * Positions to avoid deadlock

In [ ]:
def deadlock(pos, bvc, parameters, robots_path, deadlock_pointers, verbose):
    # List to return
    fixed_up_positions = []
    # Check all robots positions
    for pointer in range(len(pos)):
        # Check if the robot is in deadlock situation
        if pointer in deadlock_pointers:
            # Initialize the instantaneous set of BVC vertices
            inst_vertices = bvc["vertices"][pointer]
            # Get the sense of the vertices_bvc array
            sense = utils.get_sense(inst_vertices, robots_path[-2][pointer])
            #i = 0
            milestone = -1
            #--------------------------CONTROL POINT--------------------------
            if verbose:
                print('Robot in deadlock:', pos[pointer])
                print('Vertices to check:', inst_vertices)
                print('Sense:', sense)
            main_distance = np.linalg.norm(bvc["closer"][pointer]
                                           - inst_vertices[0])
            for i in range(len(inst_vertices)):
                temp_distance = np.linalg.norm(bvc["closer"][pointer]
                                               - inst_vertices[i])
                if verbose:
                    print('Distance from vertex to the closest point:',
                          temp_distance)
                if (temp_distance < parameters["tolerance"]
                    and temp_distance < main_distance):
                    milestone = i
                    main_distance = temp_distance
                j = 0
                for j in range(parameters["previous"]):
                    prev_dist = np.linalg.norm(inst_vertices[i]
                                               - robots_path[-2-j][pointer])
                    if verbose:
                        print('Distance from the ' + str(j)
                              + 'th previus point:', prev_dist)
                    if (prev_dist < parameters["tolerance"]
                        and prev_dist < main_distance):
                        milestone = i
                        main_distance = prev_dist
            #--------------------------CONTROL POINT--------------------------
            if verbose:
                print('Pointer:',milestone)
            if milestone > -1 and milestone < len(inst_vertices):
                #------------------------CONTROL POINT------------------------
                if verbose:
                    print('Deadlock in vertex')
                if sense == 'counterclockwise':
                    if milestone == 0:
                        aux_vertex = (inst_vertices[-2]
                                      - inst_vertices[milestone])
                    else:
                        aux_vertex = (inst_vertices[milestone-1]
                                      - inst_vertices[milestone])
                    aux_vertex = aux_vertex / np.linalg.norm(aux_vertex)
                    position_candidate =(aux_vertex*parameters["deadlock_mov"]
                                         + pos[pointer])
                elif sense == 'clockwise':
                    aux_vertex = (inst_vertices[milestone+1]
                                  - inst_vertices[milestone])
                    aux_vertex = aux_vertex / np.linalg.norm(aux_vertex)
                    position_candidate = (aux_vertex*parameters["deadlock_mov"]
                                          + pos[pointer])
            # If deadlock is on an edge
            else:
                #------------------------CONTROL POINT-------------------------
                if verbose:
                    print('Deadlock in edge')
                if sense == 'clockwise':
                    aux_vertex = bvc["closer_edges"][2*pointer] - pos[pointer]
                    aux_vertex = aux_vertex / np.linalg.norm(aux_vertex)
                    position_candidate = (aux_vertex*parameters["deadlock_mov"]
                                          + pos[pointer])
                elif sense == 'counterclockwise':
                    aux_vertex = bvc["closer_edges"][2*pointer+1]-pos[pointer]
                    aux_vertex = aux_vertex / np.linalg.norm(aux_vertex)
                    position_candidate = (aux_vertex*parameters["deadlock_mov"]
                                          + pos[pointer])
            # Make sure the point is inside BVC
            if path.Path(inst_vertices).contains_point(position_candidate):
                fixed_up_positions.append(position_candidate)
            else:
                fixed_up_positions.append(pos[pointer])
            #--------------------------CONTROL POINT---------------------------
            if verbose:
                print('-----------------------------------')
        else:
            fixed_up_positions.append(bvc["closer"][pointer])
    #------------------------------CONTROL POINT------------------------------
    return np.array(fixed_up_positions)

### Deadlock Test Function

`def deadlock_test(parameters, opt, bvc, pos, count, robots_path)`

Check if there are robots in deadlock situation

#### Parameters:
* **parameters&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;dictionary**
    * Global parameters
* **opt&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;dictionary**
    * General options
* **bvc&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;dictionary**
    * Information of BVC
* **pos&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;dictionary**
    * Robots positions and distances
* **count&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;int**
    * Step
* **robots_path&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;list of ndarryas of list of floats**
    * Previous positions
    
#### Return:
* **closer_points&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * Positions to avoid deadlock

In [ ]:
def deadlock_test(parameters, opt, bvc, pos, count, robots_path):
    deadlock_flag = False
    deadlock_pointers = []
    if count >= parameters["previous"]:
        for i in range(parameters["robots"]):
            if not np.all(bvc["closer"][i] == pos["f"][i]):
                j = 0
                deadlock_local_flag = False
                distance1 = np.linalg.norm(bvc["closer"][i] - pos["i"][i])
                if opt["verbose"]:
                    print('Distance from', opt["palette"][i],
                          'to the closest point:', distance1)
                while (not deadlock_local_flag and j < parameters["previous"]):
                    distance2 = np.linalg.norm(bvc["closer"][i]
                                               - robots_path[-2-j][i])
                    if opt["verbose"]:
                        print('Distance from the '+str(j+1)+
                              'th previous point:', distance2)
                    if (distance1 < parameters["tolerance"]
                        or distance2 < parameters["tolerance"]):
                        deadlock_pointers.append(i)
                        deadlock_local_flag = True
                        deadlock_flag = True
                    j += 1
    if opt["verbose"]:
        print('Closer positions:', bvc["closer"])
        print('Deadlock?:', deadlock_flag,'\n')
    # BREAKING DEADLOCK
    if deadlock_flag:
        if opt["verbose"]:
            print('Cells in Deadlock:')
            for i in deadlock_pointers:
                print(opt["palette"][i], end = ' ')
            print('\nCloser_edges:', bvc["closer_edges"], '\n')
        bvc["closer"] = deadlock(pos["i"], bvc, parameters,
                                        robots_path, deadlock_pointers,
                                        opt["verbose"])
        if opt["verbose"]:
            print('DEADLOCK ABOVE!')
            print('Previous positions:', robots_path[-2])
            print('New closer positions:', bvc["closer"],'\n')
    bvc.update({"in_deadlock":deadlock_pointers})
    return bvc

### Voronoi Diagram Generator

`def vor_prox(robots, vor, verbose)`

Generate the main Voronoi Diagram approximating the values of infinity vertices  
and returning the updated values of edges and vertices for BVC constructor.

#### Parameters:
* **robots&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * Robots position
* **vor&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;class scipy.spatial.Voronoi**
    * Voronoi diagram in N dimension
* **verbose&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;bool**
    * Flag for verbose
    
#### Return:
* **vertices&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * Updated vertices with approximation of infinity vertices
* **edges&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of int**
    * Updated pointers to vertices for each edge

In [40]:
def vor_prox(robots, vor, verbose):
    count = 0
    # Create copies of:
    vertices = vor.vertices
    edges = []
    # Find the center of the robots position
    center = robots.mean(axis=0)
    # pointidx: Indices of the points between which each Voronoi ridge lies.
    # simplex: Indices of the Voronoi vertices forming each Voronoi ridge.
    # zip join each values in a tuple
    for pointidx, simplex in zip(vor.ridge_points, vor.ridge_vertices):
        # Transform to array
        simplex_arr = np.array(simplex)
        # If the Voronoi edge goes to infinity
        if np.all(simplex_arr >= 0):
            # Append to the BVC edges list
            edges.append(simplex)
        # If one of the vertices goes to the infinity
        else:
            # Get the finite vertex
            i = simplex_arr[simplex_arr >= 0][0] 
            # Calculate the straight line between robots
            t = robots[pointidx[1]] - robots[pointidx[0]] 
            # Normalization of this line
            t = t / np.linalg.norm(t)
            # Ortogonalization
            n = np.array([-t[1], t[0]])
            # Get the midpoint between robots
            midpoint = robots[pointidx].mean(axis=0)
            # Far point = extension from the finite vertex using the
            #             orthogonalization and
            #             depending of the midpoint location respect center
            far_point = vor.vertices[i] + (np.sign(np.dot(midpoint- center,n))
                                           * n * 20 * len(robots))
            # Increase the count
            count += 1
            # Add the value to the BVC vertices array
            vertices = np.concatenate((vertices, far_point.reshape(1,2)),
                                      axis=0)
            # Update the pointer of BVC edges list 
            edges.append([len(vertices)-1, simplex[1]])
    if verbose:
        print('List of vertex coordinates (including approximations) of',
              'Voronoi Diagram:\n')
        for i in range(len(vertices)):
            print(i,".-",vertices[i])
        print('Robots in front: ',vor.ridge_points,'\n')
        print('Voronoi Diagram Edges: ',edges,'\n')
    return {"vrtx":vertices, "edge":edges}

### BVC Generator

`def bvc_gen(pos, radius, neighbors, vor_prox, vrtx_assembly, robots_path, opt)`

Assemble the vertices for each cell in order to generate the new BVC vertices

#### Parameters:
* **pos&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;dictionary**
    * Robots positions and distances
* **radius&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;float**
    * radius to define the BVC from the Voronoi Diagram
* **neighbors&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of ints**
    * Indices of the points between which each Voronoi ridge lies
* **vor_prox&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;dictionary**
    * Approximated values for Voronoi generation
* **vrtx_assembly&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;list of node_linker.DoublyLinkedList**
    * A list of ordered vertices for each robot
* **robots_path&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;list of ndarray**
    * Previous positions
* **opt&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;dictionary**
    * General options
    
#### Return:
* **new_robots&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * New position of robots

In [ ]:
def bvc_assembler(mode, milestone_node, vor_prox, radius, cell, bvc_vrtx, opt):
    # Auxiliar node for possible cuts
    aux_node = milestone_node.nref
    # Get coordinates
    vertex_start = vor_prox["vrtx"][milestone_node.item]
    vertex_mid1 = vor_prox["vrtx"][milestone_node.nref.item]
    vertex_mid2 = vor_prox["vrtx"][aux_node.item]
    vertex_end = vor_prox["vrtx"][aux_node.nref.item]
    # Get the new vertices in this sector
    sector = utils.intersect(vertex_start, vertex_mid1, vertex_end, radius)
    if mode == "open":
        # Assign the new values according special cases 
        if milestone_node is cell.start_node:
            bvc_vrtx.append(sector[0])
        bvc_vrtx.append(sector[1])
        if aux_node.nref.nref is None:
            bvc_vrtx.append(sector[2])
        
    elif mode == "closed":
        if opt["verbose"]:
            print(sector[1])
        bvc_vrtx.append(sector[1])
    return bvc_vrtx

In [ ]:
def bvc_plot(cell_color, opt, vrtx_assembly, bvc_vrtx, count, pos, robots_path,
             ax, closest_point, vor_prox):
    patches = []
    if opt["zoom"]:
        plt.xlim(-1.05,1.05); plt.ylim(-0.75,0.75)
        #plt.xlim(-0.53,0.53); plt.ylim(-0.38,0.38)
    else:
        plt.xlim((math.sqrt(2*len(vrtx_assembly))
                  +(1/(0.4*len(vrtx_assembly))))*-1.4,
                 (math.sqrt(2*len(vrtx_assembly))
                   +(1/(0.4*len(vrtx_assembly))))*1.4)
        plt.ylim(-(math.sqrt(2*len(vrtx_assembly))
                   +(1/(0.4*len(vrtx_assembly)))),
                  (math.sqrt(2*len(vrtx_assembly))
                   +(1/(0.4*len(vrtx_assembly)))))
    # Display edges
    for i in range(len(bvc_vrtx)-1):
        plt.plot([bvc_vrtx[i][0], bvc_vrtx[i+1][0]],
                 [bvc_vrtx[i][1], bvc_vrtx[i+1][1]],
                 color = cell_color, linestyle = '-')
    # Plot robots, target, line to target,robot path and new direction
    plt.plot(pos["i"][count,0], pos["i"][count,1], color=cell_color, marker='o')
    plt.plot(pos["f"][count,0], pos["f"][count,1], color=cell_color, marker='o')
    plt.plot([pos["i"][count,0], pos["f"][count,0]], [pos["i"][count,1],
                                                      pos["f"][count,1]],
             color = 'gray', linestyle = '--')
    # BVC Cell inside
    inst_polygon = pltpolygon(np.array(bvc_vrtx), True)
    patches.append(inst_polygon)
    p = PatchCollection(patches, alpha=0.1)
    p.set_color(cell_color)
    ax.add_collection(p)
    # Plot path
    for i in range(len(robots_path)-1):
        inst_start = robots_path[i][count]
        inst_end = robots_path[i+1][count]
        plt.plot([inst_start[0], inst_end[0]], [inst_start[1],inst_end[1]],
                 color = cell_color, linestyle = '-')
    plt.plot([pos["i"][count,0], closest_point[0]], [pos["i"][count,1],
                                                     closest_point[1]], 'k-')
    print("I am here!")
    # Display the Voronoi Diagram
    for arista in vor_prox["edge"]:
        x_values = [vor_prox["vrtx"][arista[0],0],
                    vor_prox["vrtx"][arista[1],0]]
        y_values = [vor_prox["vrtx"][arista[0],1],
                    vor_prox["vrtx"][arista[1],1]]
        plt.plot(x_values, y_values, color = 'gray', linestyle = '-')

In [2]:
def bvc_gen(pos, radius, neighbors, vor_prox, vrtx_assembly, robots_path, opt):
    count = 0
    palette = utils.colors(len(pos["i"]))
    new_robots = [] # For storing the next position of robots
    set_of_new_vertices = [] # For storing the BVC vertices of each robot
    closer_edges = [] # For save the closer edge
    # For internal color
    if opt["plot"]:
        fig, ax = plt.subplots()
    if opt["verbose"]:
        print("==================================================")
        print("                 BVC GENERATION")
        print("==================================================")
    # Calculate the bvc vertices and the closest point
    for index, cell in enumerate(vrtx_assembly):
        if opt["verbose"]:
            print('Analyzing these vertices...')
            cell.traverse_list()
        bvc_vrtx = []
        milestone_node = cell.start_node
        # Go in pairs of edges
        # Check if the cell is unlimited
        if cell.end_node.nref is None:
            if opt["verbose"]:
                print('\nOPEN CELL!')
                print('Number of vertices in this cell:',cell.lenght_list,'\n')
            while milestone_node.nref.nref is not cell.end_node.nref:
                bvc_vrtx = bvc_assembler("open", milestone_node, vor_prox,
                                         radius, cell, bvc_vrtx, opt)
                # Move the milestone_node 
                milestone_node = milestone_node.nref
            # Remove intersections of the BVC
            bvc_vrtx = utils.pruner(bvc_vrtx,opt["verbose"], "open")
        # If the cell is limited or closed
        else:
            if opt["verbose"]:
                print('\nCLOSED CELL!')
                print('Number of vertices in this cell:',cell.lenght_list)
            for _ in range(cell.lenght_list):
                bvc_vrtx = bvc_assembler("closed", milestone_node, vor_prox,
                                         radius, cell, bvc_vrtx, opt)
                # Move the milestone_node 
                milestone_node = milestone_node.nref
            # Remove intersections of the BVC
            ls = LineString(bvc_vrtx)
            lr = LineString(ls.coords[:] + ls.coords[0:1])
            if not lr.is_simple:
                if opt["verbose"]:
                    print("\nThere is an intersection!")
                mls = unary_union(lr)
                for polygon in polygonize(mls):
                    if polygon.contains(Point(pos["i"][index])):
                        break;
                bvc_vrtx = [np.array(k) for k in polygon.exterior.coords]
            else:
                bvc_vrtx.append(bvc_vrtx[0])
        # Check if target point is in/out of BVC
        bbPath = path.Path(np.array(bvc_vrtx))
        inside = bbPath.contains_point(pos["f"][count])
        if inside:
            closest_point = pos["f"][count]
            closer_edges.extend([[0,0],[0,0]])
        else:
            if opt["optimize"]:
            # Calculate the closest point using the QP-based RHC algorithm
                closer_edges, closest_point = pps.rhc(neighbors, index, pos,
                                                      opt, radius)
            # Calculate the closest point using the AGA
            else:
                closer_edges, closest_point = pps.aga(pos["i"][count],
                                                      pos["f"][count],
                                                      bvc_vrtx, closer_edges)
            if opt["verbose"]:
                print('Closest point:',closest_point)
        # Store the closest point
        new_robots.append(closest_point)
        # Save the new vertices
        set_of_new_vertices.append(bvc_vrtx)
        if opt["verbose"]:
            print('The new vertices are:',type(bvc_vrtx[0]))
            for i in range(len(bvc_vrtx)):
                print(bvc_vrtx[i])
            print('---------------------------------------------------------')
        if opt["plot"]:
            bvc_plot(palette[count], opt, vrtx_assembly, bvc_vrtx, count, pos,
                     robots_path, ax, closest_point, vor_prox)
        # Increase the counter
        count += 1
    return {"closer"       : np.array(new_robots),
            "vertices"     : np.array(set_of_new_vertices),
            "closer_edges" : np.array(closer_edges)
           }